In [38]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [14]:
train = pd.read_csv('train.csv')
result = pd.read_csv('gender_submission.csv')
test = pd.read_csv('test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
#dropping columns Cabin, Name, Ticket
train = train.drop('Cabin',1).drop('Ticket',1)
#train.head()

In [16]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [17]:
#Filling the NAN of age with average age of respective class using ('Mr','Mrs','Master','Miss') in the given candidates name
#Average Mr. Age is 32, 32
#Average Mrs. Age is 38.9, 36
#Average Master Age is 7.4,4.57
#Average Miss Age is 21.77 , 21.77
#Dr is 35
for i in range(train.shape[0]):
    if 'Master' in train['Name'][i] and np.isnan(train['Age'][i]):
        train.loc[i,'Age'] = 4 #should be based on the working of the model
        #test['Age'][i] = 7.5 
    elif 'Miss' in train['Name'][i] and np.isnan(train['Age'][i]):
        train.loc[i,'Age'] = 22 #should be based on the working of the model
    elif 'Mr.' in train['Name'][i] and np.isnan(train['Age'][i]):
        train.loc[i,'Age'] = 32 #should be based on the working of the model
    elif 'Mrs.' in train['Name'][i] and np.isnan(train['Age'][i]):
        train.loc[i,'Age'] = 36 #should be based on the working of the model
    elif 'Dr.' in train['Name'][i] and np.isnan(train['Age'][i]):
        train.loc[i,'Age'] = 35 #should be based on the working of the model    
            #test['Age'][i] = 18


In [18]:
#check for any nan in the dataset
print(sum(np.isnan(train['Age'])))
#null_data = train[train.isnull().any(axis=1)]
#print(null_data)

0


In [19]:
#merged = pd.merge(test,result,how = 'left',on = 'PassengerId')
train = train.drop('PassengerId',1)
train = train.drop('Name',1)
print(train.head())

   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S


In [23]:
train['Sex_Category'] = pd.Categorical(train.Sex).codes
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Sex_Category
0,0,3,male,22.0,1,0,7.2500,S,1
1,1,1,female,38.0,1,0,71.2833,C,0
2,1,3,female,26.0,0,0,7.9250,S,0
3,1,1,female,35.0,1,0,53.1000,S,0
4,0,3,male,35.0,0,0,8.0500,S,1


In [24]:
#remove column sex
train = train.drop('Sex',1)
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_Category
0,0,3,22.0,1,0,7.2500,S,1
1,1,1,38.0,1,0,71.2833,C,0
2,1,3,26.0,0,0,7.9250,S,0
3,1,1,35.0,1,0,53.1000,S,0
4,0,3,35.0,0,0,8.0500,S,1


In [25]:
train['Embarked_Category'] = pd.Categorical(train.Embarked).codes
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_Category,Embarked_Category
0,0,3,22.0,1,0,7.2500,S,1,2
1,1,1,38.0,1,0,71.2833,C,0,0
2,1,3,26.0,0,0,7.9250,S,0,2
3,1,1,35.0,1,0,53.1000,S,0,2
4,0,3,35.0,0,0,8.0500,S,1,2


In [27]:
train = train.drop('Embarked',1)
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_Category,Embarked_Category
0,0,3,22.0,1,0,7.2500,1,2
1,1,1,38.0,1,0,71.2833,0,0
2,1,3,26.0,0,0,7.9250,0,2
3,1,1,35.0,1,0,53.1000,0,2
4,0,3,35.0,0,0,8.0500,1,2


In [28]:
result_train = train['Survived']
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_Category,Embarked_Category
0,0,3,22.0,1,0,7.2500,1,2
1,1,1,38.0,1,0,71.2833,0,0
2,1,3,26.0,0,0,7.9250,0,2
3,1,1,35.0,1,0,53.1000,0,2
4,0,3,35.0,0,0,8.0500,1,2


In [30]:
train = train.drop('Survived',1)
train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_Category,Embarked_Category
0,3,22.0,1,0,7.2500,1,2
1,1,38.0,1,0,71.2833,0,0
2,3,26.0,0,0,7.9250,0,2
3,1,35.0,1,0,53.1000,0,2
4,3,35.0,0,0,8.0500,1,2


In [32]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                         hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(train,result_train)
clf.score(train,result_train)

0.61616161616161613

In [35]:
clf = RandomForestClassifier()
clf.fit(train,result_train)
clf.score(train,result_train)

0.96745230078563416

In [37]:
clf = LogisticRegression()
clf.fit(train,result_train)
clf.score(train,result_train)

0.80134680134680136

In [40]:
clf = SVC()
clf.fit(train,result_train)
clf.score(train,result_train)

0.90684624017957349

In [89]:
test = pd.read_csv('test.csv')

In [94]:
Master = test[test.Name  "Master"]
Master

KeyError: False

In [41]:
#Random Forest seems to do the best job
#Pre-Processing testing data

test = test.drop('Cabin',1).drop('Ticket',1)

for i in range(test.shape[0]):
    if 'Master' in test['Name'][i] and np.isnan(test['Age'][i]):
        test.loc[i,'Age'] = 4 #should be based on the working of the model
        #test['Age'][i] = 7.5 
    elif 'Miss' in test['Name'][i] and np.isnan(test['Age'][i]):
        test.loc[i,'Age'] = 22 #should be based on the working of the model
    elif 'Mr.' in test['Name'][i] and np.isnan(test['Age'][i]):
        test.loc[i,'Age'] = 32 #should be based on the working of the model
    elif 'Mrs.' in test['Name'][i] and np.isnan(test['Age'][i]):
        test.loc[i,'Age'] = 36 #should be based on the working of the model
    elif 'Dr.' in test['Name'][i] and np.isnan(test['Age'][i]):
        test.loc[i,'Age'] = 35 #should be based on the working of the model    
            #test['Age'][i] = 18
#check for any nan in the dataset
print(sum(np.isnan(test['Age'])))

1


In [85]:
# pd.where(np.isnan(test['Age']) == True)
#test.where(test['Age'])
print(test[(test.Age.isnull())])
print(test[(test.Fare.isnull())])

    PassengerId  Pclass                     Name     Sex  Age  SibSp  Parch  \
88          980       3  O'Donoghue, Ms. Bridget  female  NaN      0      0   

    Fare Embarked  
88  7.75        Q  
     PassengerId  Pclass                Name   Sex   Age  SibSp  Parch  Fare  \
152         1044       3  Storey, Mr. Thomas  male  60.5      0      0   NaN   

    Embarked  
152        S  


NameError: name 'avg' is not defined